In [15]:
import tensorflow as tf
from PIL import Image
import numpy as np
from datetime import datetime


#이미지, 상수들
learning_rate=1e-3
W1=640
H1=360
W2=1280
H2=720
path=""
pref1="360p/"
pref2="720p/"
suff1="_360.jpg"
suff2="_720.jpg"
train_num=30
file_num=8#6#30
#batch_num=1000


#가중치 초기화 함수
def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial, name=name)
#절편 초기화 함수
def bias_variable(shape, name):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial, name=name)
#2D 컨벌루션 실행
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def getimage(idx):
    img_1=Image.open(path+pref1+str(idx)+suff1)
    array_1=np.array(img_1)[:, :]
    array_1=array_1.astype(np.float32)

    img_1_720 = img_1.resize((W2, H2))
    array_1_720=np.array(img_1_720)[:, :]
    array_1_720=array_1_720.astype(np.float32)

    img_2=Image.open(path+pref2+str(idx)+suff2)
    array_2=np.array(img_2)[:, :, 0:3]
    array_2=array_2.astype(np.float32)
    return array_1, array_1_720, array_2

def l_relu(x, alpha=0.):
    return tf.nn.relu(x)-alpha*tf.nn.relu(-x)



#학습때 사용하는 변수들
x_image = tf.placeholder(np.float32, shape=[None, H1, W1, 3])
x_image_720 = tf.placeholder(np.float32, shape=[None, H2, W2, 3])
y_image = tf.placeholder(np.float32, shape=[None, H2, W2, 3])
#가중치, 절편, 결과
W_conv = weight_variable([5, 5, 3, 3], name='weight')##
b_conv = bias_variable([3], name='bias')##
y_conv = l_relu(conv2d(x_image_720, W_conv)+b_conv, alpha=0.5)##
y_res = tf.reshape(y_conv, [-1, H2, W2, 3])+x_image_720# 고쳐야됨

cost = tf.reduce_sum((y_image-y_res)*(y_image-y_res))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cost)
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#saver.restore(sess, "01/models.ckpt")


for steps in range(train_num):
    for index in range(460, 460+file_num):
        array360, array360_720, array720 = getimage(index)
        sess.run(train_step, feed_dict={x_image:[array360], x_image_720:[array360_720], y_image:[array720]})
    #print(sess.run(cost, feed_dict={x_image:[array360], y_image:[array720]}))
    if(steps%5==4):
        showres(index, steps)
    if(steps%10==0):
        print(sess.run(cost, feed_dict={x_image:[array360], x_image_720:[array360_720],y_image:[array720]}))
        #f=open("C:/image/logs.txt", "a")
        #f.write(str(datetime.now())+" | "+str(steps)+" :  "+str(sess.run(cost, feed_dict={x_image:[array360], y_image:[array720]}))+'\n')
        #f.close()
    #if(steps%10==):
        #save_path=saver.save(sess, "01/models.ckpt")
print ("끝났띠")

2.78077e+08
1.46634e+08
1.31642e+08
끝났띠


In [14]:
def showres(index, steps):
    test360, test_res, test720 = getimage(index)
    A=sess.run(y_res, feed_dict={x_image:[test360], x_image_720:[test_res], y_image:[test720]})
    result = A[0].astype(np.uint8)
    img360=test360.astype(np.uint8)
    img720=test720.astype(np.uint8)
    #Image.fromarray(img360, 'RGB').save('res/img360_'+str(steps)+'.jpg')
    #Image.fromarray(img720, 'RGB').save('res/img720_'+str(steps)+'.jpg')
    Image.fromarray(result, 'RGB').save('res/result_'+str(steps)+'.jpg')